In [ ]:
# notebooks/churn_analysis.ipynb
import pandas as pd
from src.utils import load_and_preprocess_data, perform_eda




In [ ]:
# Load dataset and perform EDA
file_path = '../data/bank_churn.csv'
df = load_and_preprocess_data(file_path)
perform_eda(df)